In [49]:
from dataclasses import dataclass
from typing import Sequence

@dataclass
class Term: pass

@dataclass
class Var(Term):
    name: str

Environment: TypeAlias = Sequence[tuple[Var,Term]]

@dataclass
class Fun(Term):
    x: Var
    t: Term

@dataclass
class App(Term):
    l: Term
    r: Term

@dataclass
class Value(Term): pass

@dataclass
class Num(Value):
    n: int

@dataclass
class Op(Term):
    op: str
    left: Term
    right: Term

@dataclass
class Ifz(Term):
    cond: Term
    thenTerm: Term
    elseTerm: Term

@dataclass
class Fix(Term):
    x: Var
    t: Term

@dataclass
class Let(Term):
    x: Var
    t: Term
    body: Term

@dataclass
class Closure(Value):
    x: Var
    t: Term
    e: Environment

@dataclass
class Thunk(Term):
    term: Fix
    e: Environment

def searchEnv(x: Var, e: Environment) -> Value:
    for (var, val) in e:
        if var == x:
            match val:
                case Thunk(term, e2):
                    return interp(term, e2)
                case _: return val
    raise Exception("Unbound variable: " + str(x))

def extendEnv(x: Var, t: Term, e: Environment) -> Environment:
    return [(x, t)] + e

def checkNumber(num: Value) -> int:
    match num:
        case Num(n): return n
    raise Exception("Not a number: " + str(num))

def interp(t: Term, e: Environment) -> Value:
    match t:
        case Var(x): return searchEnv(t, e)
        case App(t, u):
            w = interp(u, e)
            match interp(t, e):
                case Closure(x, t2, e2):
                    return interp(t2, extendEnv(x, w, e2))
                case _: raise Exception("Illegal function: " + str(v))
        case Fun(x, t2): return Closure(x, t2, e)
        case Num(n): return t
        case Op(op, l, r):
            lv = checkNumber(interp(l, e))
            rv = checkNumber(interp(r, e))
            match op:
                case '+': return Num(lv + rv)
                case '-': return Num(lv - rv)
                case '*': return Num(lv * rv)
                case '/': return Num(lv / rv)
                case '%': return Num(lv % rv)
                case '<': return Num(1 if lv < rv else 0)
                case '=': return Num(1 if lv == rv else 0)
                case _: raise Exception("Unknown op: " + str(op))
        case Ifz(cond, t, u):
            c = interp(cond, e)
            match c:
                case Num(0): return interp(t, e)
                case Num(_): return interp(u, e)
                case _: raise Exception("Condition not a number: " + str(c))
        case Fix(x, t2):
            return interp(t2, extendEnv(x, Thunk(t, e), e))
        case Let(x, t, u):
            w = interp(t, e)
            return interp(u, extendEnv(x, w, e))

print(interp(Op('+', Num(1), Num(2)), []))  # Numeric expression
print(interp(Var('x'), [(Var('x'), Num(1))]))  # Variable
print(interp(App(Fun(Var('x'), Op('*', Var('x'), Var('x'))), Num(3)), [])) # Function and application
print(interp(App(Fix(Var('f'), Fun(Var('x'), Ifz(Var('x'), Num(1), Op('*', Var('x'), App(Var('f'), Op('-', Var('x'), Num(1))))))), Num(10)), [])) # Recursive function call using Fix


Num(n=3)
Num(n=1)
Num(n=9)
Num(n=3628800)


In [47]:
@dataclass
class FixFun(Term):
    f: Var
    x: Var
    t: Term

@dataclass
class RecClosure(Value):
    f: Var
    x: Var
    t: Term
    e: Environment

def searchEnv(x: Var, e: Environment) -> Value:
    for (var, val) in e:
        if var == x:
            return val
    raise Exception("Unbound variable: " + str(x))

def interp(t: Term, e: Environment) -> Value:
    match t:
        case Var(x): return searchEnv(t, e)
        case App(t, u):
            w = interp(u, e)
            v = interp(t, e)
            match v:
                case RecClosure(f, x, t2, e2):
                    return interp(t2, extendEnv(x, w, extendEnv(f, v, e2)))
                case _: raise Exception("Illegal function: " + str(v))
        case Fun(x, t2): return RecClosure(Var(''), x, t2, e)
        case Num(n): return t
        case Op(op, l, r):
            lv = checkNumber(interp(l, e))
            rv = checkNumber(interp(r, e))
            match op:
                case '+': return Num(lv + rv)
                case '-': return Num(lv - rv)
                case '*': return Num(lv * rv)
                case '/': return Num(lv / rv)
                case '%': return Num(lv % rv)
                case '<': return Num(1 if lv < rv else 0)
                case '=': return Num(1 if lv == rv else 0)
                case _: raise Exception("Unknown op: " + str(op))
        case Ifz(cond, t, u):
            c = interp(cond, e)
            match c:
                case Num(0): return interp(t, e)
                case Num(_): return interp(u, e)
                case _: raise Exception("Condition not a number: " + str(c))
        case FixFun(f, x, t2):
            return RecClosure(f, x, t2, e)
        case Let(x, t, u):
            w = interp(t, e)
            return interp(u, extendEnv(x, w, e))

print(interp(App(FixFun(Var('f'), Var('x'), Ifz(Var('x'), Num(1), Op('*', Var('x'), App(Var('f'), Op('-', Var('x'), Num(1)))))), Num(10)), [])) # Recursive function call using FixFun


Num(n=3628800)
